In [1]:
%cd ..

import pandas as pd
pd.set_option('display.max_columns', None)

from scripts.utils_latest import *

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
def find_least_different_string(original_string, column, cache=None):
    if cache is None:
        cache = {}
    min_distance = float('inf')
    least_different_string = ''
    for sequence in column:
        if sequence in cache:
            distance = cache[sequence]
        else:
            distance = editdistance.eval(original_string, sequence)
            cache[sequence] = distance
        if distance < min_distance:
            min_distance = distance
            least_different_string = sequence
    return least_different_string


In [3]:
# get dfs
df = pd.read_csv("results/7_results_of_first_4_rows.csv")
targetscan = pd.read_csv("data/processed/targetscan/targetscan.csv").rename(columns={"accession": "mirna_accession", "conservation": "mirna_conservation"})

# get mirnas without conservation
big_mirnas = df.mirna_accession.unique()
small_mirnas = targetscan.mirna_accession.unique()

diff = set(big_mirnas) - set(small_mirnas)
mask = df.mirna_accession.isin(diff)

# assigning to a df
conservations_to_calculate = (df[mask][["mirna_accession", "mirna_sequence"]]
                            .assign(mirna_conservation=None)
                            .assign(seed=df.mirna_sequence.str[1:7])
                            .rename(columns={"mirna_sequence": "sequence"}))


# append least difference string column
for i, row in conservations_to_calculate.iterrows():
    
    original_string = row["sequence"]
    
    target_sequences = targetscan.sequence
    
    least_different_string = find_least_different_string(original_string, target_sequences)
    
    conservations_to_calculate.loc[i, "least_different_string"] = least_different_string
    

# getting seq:conservation pairs from targetscan
pair_dict = dict(zip(targetscan['sequence'], targetscan['mirna_conservation']))

# appending conservation values of least_different_strings
conservations_to_calculate['conservation'] = conservations_to_calculate['least_different_string'].map(pair_dict)

